# Programación orientada a objetos

Ya hemos discutido sobre los tipos de colecciones en Python, funcionan mas o menos así:
1. El tipo puede ser llamado como función para crear nuevas **instancias**.
2. La llamada al tipo puede incluir argumentos usados para inicializar la nueva instancia.
3. Podemos invocar **métodos** sobre instancias de los tipos en lugar de inspeccionar y manipular por su representación interna.
4. Métodos para distintos tipos pueden tener el mismo nombre.
5. La forma en que se imprime una instancia se basa en su tipo.

Nada de lo anterior es especial para los tipos de colecciones, en realidad, todas estas características son parte de un mecanismo incorporado a Python para organizar ideas y modelar problemas.

Podemos crear un nuevo tipo en Python con una definición de **clase**, similar a como podemos definir una nueva función.

Veamos un ejemplo y luego discutamos algunas propiedades de estas técnicas de programación.

In [3]:
class Problem:
    """Esta es una clase abstracta para un problema.
    No se supone que debamos crear objetos de esta
    clase directamente. La idea es que otras clases
    hereden de esta para crear clases de problemas
    más especeificos."""
    
    def __init__(self, initial, goal=None):
        """Esta definición es un método, es especial
        porque comienza y termina con doble piso.
        En Python, el método __init__ es utilizado
        para construir objetos de la clase.
        En este y el resto de los métodos, el primer
        parámetro es self, y nos permite referirnos
        al objeto sobre el cuál el método es invocado."""
        self.initial = initial
        self.goal = goal
    
    def actions(self, state):
        """Un problema contiene un estado inicial y
        quizá una meta en particular. El método actions
        debe regresar las acciones que se pueden tomar
        en el estado dado. El resultado usualmente es
        una lista, pero si debes regresar muchas acciones,
        considera usar yield para generarlas una a la vez."""
        raise NotImplementedError
    
    def result(self, state, action):
        """El método anterior, al igual que este, tiene una
        implementación trivial. Simplemente señalamos un
        error que indica que no hay una implementación
        para el método. En una implementación de este método,
        debemos regresar el estado al que llegamos después
        de ejecutar la acción dada en el estado dado.
        La acción debe ser una calculada por
        self.actions(state)."""
        raise NotImplementedError
    
    def goal_test(self, state):
        """De seguro ya te diste cuenta que todos estos
        métodos son muy generales y poco específicos.
        ¿Qué forma tienen los estados?
        ¿Qué son las acciones?
        ¿Qué es una meta?
        Este método regresa True si el estado dado es una
        meta. Por defecto se compara el estado dado con
        self.goal o verifica que el estado dado sea
        elemento de self.goal en caso de ser lista."""
        if isinstance(self.goal, list):
            return any(x is state for x in self.goal)
        return state == self.goal
    
    def path_cost(self, c, state1, action, state2):
        """El método anterior si tenía una implementación útil,
        sigue siendo general, pero podemos pensar que en muchos
        problemas, si un estado es igual a la meta, entonces hemos
        llegado a la meta. El método path_cost también incluye una
        implementación: Se considera que llegar al estado 1 tiene
        un costo c. Regresa el costo de un camino de solución que
        llega al estado 2 desde estado 1 por medio de la acción dada."""
        return c + 1
    
    def value(self, state):
        """En distintos problemas, cada estado tiene un valor asociado.
        Este método sirve para obtener este valor."""
        raise NotImplementedError

In [4]:
# Practicando...
def checa_is(a,b):
    return a is b, a == b

checa_is(1,1)

xs = (1,2,3)
ys = (1,2,3)

checa_is([1,2,3],[1,2,3])

checa_is(xs,ys)
# Son 2 listas iguales pero no son el mismo objeto...
# is devuelve True si son el mismo objeto o si son inmutables en la mayoría... 

(False, True)

En el ejemplo anterior podemos observar distintos aspectos de la programación orientada a objetos. 

Primero, observemos que un problema puede ser especificado a partir de un estado inicial y un(os) estado(s) objetivo(s). Cuando se trabajan con objetos que son instancias de esta clase, podremos consultar su estado inicial y su(s) objetivo(s), estos valores particulares viven **dentro** del objeto y forman parte de su identidad.

Al crear un objeto de una clase, se crea un nuevo espacio de nombres, por lo tanto si `x` y `y` son dos problemas distintos, nos referimos a sus estados iniciales de la misma forma `x.initial` y `y.initial`.

Segundo, observamos que una clase no solo nos informa sobre los nombres que conforman la estructura de un objeto problema. Considera los siguientes incisos:
- En *este* estado, ¿Qué acciones puedo realizar? `Problem.actions`
- Si estoy en *este* estado y realizo *esta* acción, ¿A qué estado llego? `Problem.result`
- Me encuentro en *este* estado, ¿Ya llegué a la meta? `Problem.goal_test`
- Fue *así* de costoso llegar a *este* estado, si realizo *esta* acción y llego a *este otro* estado, ¿Qué tan costoso será? `Problem.path_cost`

Estos métodos nos proveen todo un vocabulario para comunicar ideas, razonar y eventualmente resolver problemas.

Tercero, la orientación a objetos nos permite *modelar* ciertos problemas identificando qué tienen en común, qué tienen diferente, e incluso si un problema es parte de otro.

La idea de este paradigma es construir tus programas como objetos relacionados entre sí. Estos objetos se podrán comunicar a partir de sus métodos y estos serán cuidadosamente diseñados para contemplar las necesidades de la mayoría de los objetivos específicos de nuestro programa.

In [5]:
class Node:
    """Esta es la clase nodo. Nos permite modelar un árbol
    donde los vertices son estados y las aristas son acciones.
    Al resolver un problema, podemos trabajar con nodos para
    representar el camino que se ha tomado hasta llegar a la
    solución."""
    
    def __init__(self, state, parent=None, action=None, path_cost=0):
        """Crea un nodo, a partir de (posiblemente) otro nodo padre
        y una acción."""
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1
    
    def __repr__(self):
        """Ver ejemplo más adelante"""
        return f"<Node {self.state}>"
    
    def __lt__(self, node):
        """Ver ejemplo más adelante"""
        return self.state < node.state
    
    def __eq__(self, other):
        """Ver ejemplo más adelante"""
        return (
            isinstance(other, Node) and
            self.state == other.state
        )
    
    def __hash__(self):
        """Ver ejemplo más adelante"""
        return hash(self.state)
    
    def expand(self, problem):
        """Enlista los nodos que son alcanzables con una acción
        a partir de este nodo."""
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]
    
    def child_node(self, problem, action):
        """Construye un nodo hijo a partir de efectuar una acción en
        el estado actual."""
        next_state = problem.result(self.state, action)
        next_cost = problem.path_cost(self.path_cost, self.state, action, next_state)
        next_node = Node(next_state, self, action, next_cost)
        return next_node
    
    def solution(self):
        """Regresa una lista de acciones para llegar a este nodo."""
        return [node.action for node in self.path()[1:]]
    
    def path(self):
        """Regresa una lista de nodos que forman la trayectoria desde
        la raíz hasta este nodo."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))
    def __str__(self):
        return "<Node>"

## Ejemplo

In [6]:
class HomoSapiens:
    def __init__(self, nombre, edad):
        self.nombre = nombre
        self.edad = edad

class Persona(HomoSapiens):
    def __init__(self, nombre, edad):
        super().__init__(nombre, edad)
    
    def __repr__(self):
        return f"<Persona {self.nombre}>"
    
    def __lt__(self, otra):
        return self.nombre < otra.nombre
    
    def __eq__(self, otra):
        return (
            isinstance(otra, HomoSapiens) and
            self.nombre == otra.nombre
        )
    
    def __hash__(self):
        return hash(self.nombre)

In [7]:
HomoSapiens('Eduardo', 29)

In [8]:
x = HomoSapiens('Eduardo', 30)
y = HomoSapiens('Eduardo', 30)

In [9]:
x == y

False

In [10]:
# x < y

In [11]:
hash(x)

156342502785

In [12]:
hash(y)

156342938037

In [13]:
hash(x) == hash(y)

False

In [14]:
d = {}
d[x] = 1
d[y] = 2
d

{<__main__.HomoSapiens at 0x2466bd45810>: 1,
 <__main__.HomoSapiens at 0x2466c3e9b50>: 2}

In [15]:
Persona('Eduardo', 29)

<Persona Eduardo>

In [16]:
x = Persona('Eduardo', 29)
y = Persona('Eduardo', 29)

In [17]:
x == y

True

In [18]:
x < y

False

In [19]:
hash(x)

7967756951457964803

In [20]:
hash(y)

7967756951457964803

In [21]:
hash(x) == hash(y)

True

In [22]:
d = {}
d[x] = 'Hola'
d[y] = 'Adios'
d

{<Persona Eduardo>: 'Adios'}

In [23]:
x = Persona('Eduardo', 29)
y = Persona('Eduardo', 3)

In [24]:
x == y

True

In [25]:
x < y

False

In [26]:
hash(x)

7967756951457964803

In [27]:
hash(y)

7967756951457964803

In [28]:
hash(x) == hash(y)

True

In [29]:
class Abstracta:
    def __init__(self, x):
        self.x = x
    
    def foo(self):
        raise NotImplementedError

class Concreta1(Abstracta):
    def __init__(self, x):
        super().__init__(x)
        self.x = 2 * self.x
    
    def foo(self):
        print(f"Foo fooo!!!! valor de x es = {self.x}")

class Concreta2(Abstracta):
    def __init__(self, x, y):
        super().__init__(x)
        self.x = self.x + y
    
    def foo(self):
        print(f"Fi fa fo fú!!! valor de x es = {self.x}")

def llama_foo(lista_de_objetos_concretos):
    for x in lista_de_objetos_concretos:
        x.foo()

In [30]:
llama_foo([
    Concreta1(4),
    Concreta2(5, 1),
    Concreta2(0, 0),
    Concreta1(12),
])

Foo fooo!!!! valor de x es = 8
Fi fa fo fú!!! valor de x es = 6
Fi fa fo fú!!! valor de x es = 0
Foo fooo!!!! valor de x es = 24


# Resolución de problemas genéricos reciclables

## El problema

![GIF vuelo](https://media.giphy.com/media/3o6nV8OYdUhiuKja1i/giphy.gif)

In [218]:
import time
import random
import math

Planear un viaje para un grupo de personas que viven en distintos lugares llegando al mismo lugar siempre es un reto.

Consideremos que los miembros de una familia vienen de todas partes de Estados Unidos para encontrarse en Nueva York. Todos van a llegar el mismo día y regresarse el mismo día, y quisieran compartir transporte hacia y desde el aeropuerto.

Definimos una lista con parejas (*Nombre*, *Aeropuerto*) que indican el aeropuerto del que parte cada miembro de la familia.

In [219]:
people = [('Seymour', 'BOS'),
          ('Franny', 'DAL'),
          ('Zooey', 'CAK'),
          ('Walt', 'MIA'),
          ('Buddy', 'ORD'),
          ('Les', 'OMA')]

In [220]:
destination = 'LGA'


Podemos obtener información de cada aeropuerto a partir del archivo `airport-codes.txt`.

In [221]:
def load_airports(path):
    airports = {}
    with open(path) as f:
        f.readline()
        for line in f:
            cols = line.strip().split(',')
            iata = cols[9]
            name = cols[2]
            region = cols[6]
            municipality = cols[7]
            airports[iata] = (
                name,
                region,
                municipality
            )
    return airports

In [222]:
airports = load_airports('assets/airport-codes.txt')

Veamos una descripción del viaje de ida de cada miembro de la familia

In [223]:
for name, iata in people:
    print(f"\
* {name} parte de \
“{airports[iata][0]}” en {airports[iata][2]} \
y viaja hasta \
“{airports[destination][0]}” en {airports[destination][2]}\
")

* Seymour parte de “General Edward Lawrence Logan International Airport” en Boston y viaja hasta “La Guardia Airport” en New York
* Franny parte de “Dallas Love Field” en Dallas y viaja hasta “La Guardia Airport” en New York
* Zooey parte de “Akron Canton Regional Airport” en Akron y viaja hasta “La Guardia Airport” en New York
* Walt parte de “Miami International Airport” en Miami y viaja hasta “La Guardia Airport” en New York
* Buddy parte de “Chicago O'Hare International Airport” en Chicago y viaja hasta “La Guardia Airport” en New York
* Les parte de “Eppley Airfield” en Omaha y viaja hasta “La Guardia Airport” en New York


Existen muchos vuelos al día para llegar a Nueva York desde las ubicaciones de los miembros de esta familia, todos saliendo a distintos tiempos y con distinto precio.

Podemos obtener información de los vuelos disponibles a partir del archivo `schedule.txt`.

In [224]:
def load_flights(path):
    flights = {}
    with open(path) as f:
        for line in f:
            origin, dest, t_depart, t_arrive, price = line.strip().split(',')
            flights.setdefault((origin, dest), [])
            flights[(origin, dest)].append((t_depart, t_arrive, int(price)))
    return flights

In [225]:
flights = load_flights('assets/schedule.txt')

In [226]:
type(flights)
flights


{('LGA', 'OMA'): [('6:19', '8:13', 239),
  ('8:04', '10:59', 136),
  ('9:31', '11:43', 210),
  ('11:07', '13:24', 171),
  ('12:31', '14:02', 234),
  ('14:05', '15:47', 226),
  ('15:07', '17:21', 129),
  ('16:35', '18:56', 144),
  ('18:25', '20:34', 205),
  ('20:05', '21:44', 172)],
 ('OMA', 'LGA'): [('6:11', '8:31', 249),
  ('7:39', '10:24', 219),
  ('9:15', '12:03', 99),
  ('11:08', '13:07', 175),
  ('12:18', '14:56', 172),
  ('13:37', '15:08', 250),
  ('15:03', '16:42', 135),
  ('16:51', '19:09', 147),
  ('18:12', '20:17', 242),
  ('20:05', '22:06', 261)],
 ('LGA', 'ORD'): [('6:03', '8:43', 219),
  ('7:50', '10:08', 164),
  ('9:11', '10:42', 172),
  ('10:33', '13:11', 132),
  ('12:08', '14:47', 231),
  ('14:19', '17:09', 190),
  ('15:04', '17:23', 189),
  ('17:06', '20:00', 95),
  ('18:33', '20:22', 143),
  ('19:32', '21:25', 160)],
 ('ORD', 'LGA'): [('6:05', '8:32', 174),
  ('8:25', '10:34', 157),
  ('9:42', '11:32', 169),
  ('11:01', '12:39', 260),
  ('12:44', '14:17', 134),
  ('14

Podemos consultar los posibles vuelos de ida para un miembro de la familia (en este ejemplo el primero en la lista, que es Seymour) de la siguiente forma:

In [227]:
flights[(
    people[0][1],
    destination,
)]

[('6:17', '8:26', 89),
 ('8:04', '10:11', 95),
 ('9:45', '11:50', 172),
 ('11:16', '13:29', 83),
 ('12:34', '15:02', 109),
 ('13:40', '15:37', 138),
 ('15:27', '17:18', 151),
 ('17:11', '18:30', 108),
 ('18:34', '19:36', 136),
 ('20:17', '22:22', 102)]

Para su vuelo de regreso, simplemente cambiamos el orden del viaje:

In [228]:
flights[(
    destination,
    people[0][1],
)]

[('6:39', '8:09', 86),
 ('8:23', '10:28', 149),
 ('9:58', '11:18', 130),
 ('10:33', '12:03', 74),
 ('12:08', '14:05', 142),
 ('13:39', '15:30', 74),
 ('15:25', '16:58', 62),
 ('17:03', '18:03', 103),
 ('18:24', '20:49', 124),
 ('19:58', '21:23', 142)]

Una desventaja de nuestra actual representación para los vuelos es la forma en que se especifican los tiempos.

Para trabajar con valores numéricos, vamos a considerar una función que calcula la cantidad de minutos que han pasado en el día para cierta hora.

In [229]:
import time

def get_minutes(t):
    x = time.strptime(t, '%H:%M')
    h = x.tm_hour
    m = x.tm_min
    return 60 * h + m

In [230]:
get_minutes('23:50')

1430

In [231]:
get_minutes('00:51')

51

In [232]:
get_minutes('01:12')

72

In [233]:
list(
    map(
        lambda x: ( # Función anónima Lambda...
            get_minutes(x[0]),
            get_minutes(x[1]),
            x[2],
        ),
        flights[(
            people[0][1],
            destination
        )]
    )
)

[(377, 506, 89),
 (484, 611, 95),
 (585, 710, 172),
 (676, 809, 83),
 (754, 902, 109),
 (820, 937, 138),
 (927, 1038, 151),
 (1031, 1110, 108),
 (1114, 1176, 136),
 (1217, 1342, 102)]

In [234]:
list(map(lambda x: (get_minutes(x[0]), get_minutes(x[1]), x[2]),flights[(people[0][1],destination)]))

[(377, 506, 89),
 (484, 611, 95),
 (585, 710, 172),
 (676, 809, 83),
 (754, 902, 109),
 (820, 937, 138),
 (927, 1038, 151),
 (1031, 1110, 108),
 (1114, 1176, 136),
 (1217, 1342, 102)]

In [235]:
people

[('Seymour', 'BOS'),
 ('Franny', 'DAL'),
 ('Zooey', 'CAK'),
 ('Walt', 'MIA'),
 ('Buddy', 'ORD'),
 ('Les', 'OMA')]

In [236]:
a = flights[("LGA", people[0][1])]
print(random.choice(a))

('17:03', '18:03', 103)


Vamos a considerar una estructura particular para las posibles soluciones al problema. Una representación usual es que las soluciones sean listas de números. En nuestro caso, cada número puede representar el vuelo de ida o el vuelo de regreso, de tal manera que el tamaño de la solución es dos veces más que la cantidad de personas.

Por ejemplo,
```
[1,4,3,2,7,3,6,3,2,4,5,3]
```

Nos representa una solución donde:
- la persona con índice `0` toma el vuelo de ida con índice `1` y el vuelo de regreso con índice `4`,
- la persona con índice `1` toma el vuelo de ida con índice `3` y el vuelo de regreso con índice `2`,
- la persona con índice `3` toma el vuelo de ida con índice `7` y el vuelo de regreso con índice `3`,
- etc.

El índice de las personas hace referencia a la lista `people`, si dicha persona vive en `A` y va a `B`, entonces el índice de ida hace referencia a la lista `flights[(A,B)]`, mientras que el índice de regreso hace referencia a la lista `flights[(B,A)]`.

Todo esto puede sonar confuso, lo conveniente es elegir una representación lo suficientemente simple para que nuestros programas encuentren buenas soluciones, pero lo suficientemente complejo como para entender soluciones como personas.

La siguiente función nos permite tomar un valor de solución e imprimir la información de forma legible.

In [237]:
def print_schedule(s):
    for i in range(len(s) // 2):
        name = people[i][0]
        origin = people[i][1]
        out = flights[(origin,destination)][s[2*i]]
        ret = flights[(destination,origin)][s[2*i+1]]
        print('%10s%10s %5s-%5s $%3s %5s-%5s $%3s' % (
            name,
            airports[people[i][1]][2],
            out[0],out[1],out[2],
            ret[0],ret[1],ret[2],
        ))

In [238]:
# Soluición aleatoria:
import random


def random_sol():
    solution_r=[]
    for i in range(len(people)):
        opciones_ida = flights[(people[i][1], "LGA")]
        opciones_regreso = flights[("LGA",people[i][1])]
        
        ida = random.randint(0, len(opciones_ida) - 1)
        regreso = random.randint(0, len(opciones_regreso) - 1)
        
        solution_r.append(ida)
        solution_r.append(regreso)
    
    return solution_r

a = random_sol()

print(a)

print_schedule(a)

[7, 0, 2, 6, 5, 1, 7, 8, 1, 1, 7, 7]
   Seymour    Boston 17:11-18:30 $108  6:39- 8:09 $ 86
    Franny    Dallas  9:08-12:12 $364 15:49-20:10 $497
     Zooey     Akron 13:40-15:38 $137  8:19-11:16 $122
      Walt     Miami 17:07-20:04 $291 18:07-21:30 $355
     Buddy   Chicago  8:25-10:34 $157  7:50-10:08 $164
       Les     Omaha 16:51-19:09 $147 16:35-18:56 $144


In [239]:
print_schedule([1,4,3,2,7,3,6,3,2,4,5,3])

   Seymour    Boston  8:04-10:11 $ 95 12:08-14:05 $142
    Franny    Dallas 10:30-14:57 $290  9:49-13:51 $229
     Zooey     Akron 17:08-19:08 $262 10:32-13:16 $139
      Walt     Miami 15:34-18:11 $326 11:08-14:38 $262
     Buddy   Chicago  9:42-11:32 $169 12:08-14:47 $231
       Les     Omaha 13:37-15:08 $250 11:07-13:24 $171


Hasta ahora, hemos modelado este problema de forma computacional y tenemos codificaciones prácticas para razonar sobre personas, vuelos y tiempos.
Sin embargo, podemos ver que la solución de ejemplo nos dice que Walt viaja desde Miami en el vuelo que parte a las 15:34 y regresa el mismo dia a las 11:08...

Esta es una pésima solución... ¡es imposible!

Para poder avanzar, necesitamos un criterio que nos permita comparar dos soluciones, claramente la que del ejemplo es pésima, pero ¿De qué forma podemos especificar que una solución es mejor que esta?

## Función de costo

![GIF costoso](https://media.giphy.com/media/yIxNOXEMpqkqA/giphy.gif)

Vamos a definir una *función de costo*, de tal manera que el problema se va a reducir a encontrar un conjunto de entradas (vuelos en este caso) que minimice la función de costo, es decir, la que tenga el costo más bajo.

Esta función de costo debe recibir una posible solución y darnos un valor numérico que nos indique qué tan mala es.

Suele ser dificil determinar qué hace que una solución sea buena o mala cuando se involucran varios factores. Consideremos algunos candidatos para este problema:

- Precio: El precio total de todos los vuelos de avión, o posiblemente un promedio ponderado que tome en cuenta la situación financiera de los familiares.
- Tiempo de viaje: El tiempo total de viaje para todos los familiares.
- Tiempo de espera: El tiempo que van a esperar en el aeropuerto a que lleguen los demás miembros de la familia.
- Tiempo de salida: Los vuelos que salen demasiado temprano pueden imponer costo adicional a los pasajeros que no durmieron plenamente en la noche por llegar al aeropuerto.
- Periodo de renta de carro: Si la familia renta un carro, deben regresarlo lo suficientemente temprano en el día para que no les cobren un día adicional.

![GIF cansado](https://media.giphy.com/media/AgP9GXdpREaURYbxca/giphy.gif)

Podemos imaginarnos otros factores que tomar en cuenta para nuestro problema particular.

Una vez que determinamos el conjunto de factores que afectan nuestra noción de *costo*, debemos determinar cómo combinarlas en un número.

In [240]:
a = random_sol()
a

[1, 9, 2, 2, 7, 5, 5, 1, 5, 2, 7, 6]

In [241]:
def schedule_cost(s):
    # contamos el precio total de cada vuelo (ida y regreso)
    total_price = 0
    
    # nos interesa conocer el tiempo de llegada a NY mas tarde
    # y el tiempo de salida de NY mas temprano.
    latest_arrival = 0
    earliest_departure = 24 * 60
    
    for i in range(len(s) // 2):
        origin = people[i][1]
        out_flight = flights[(origin, destination)][s[2*i]]
        ret_flight = flights[(destination, origin)][s[2*i+1]]
        
        total_price += out_flight[2] # vuelo de ida
        total_price += ret_flight[2] # vuelo de regreso
        
        # tiempo de llegada máximo
        # tiempo de salida mínimo
        if latest_arrival < get_minutes(out_flight[1]):
            latest_arrival = get_minutes(out_flight[1])
        if earliest_departure > get_minutes(ret_flight[0]):
            earliest_departure = get_minutes(ret_flight[0])
    
    # contamos el tiempo de espera de cada persona
    total_wait = 0
    
    for i in range(len(s) // 2):
        origin = people[i][1]
        out_flight = flights[(origin, destination)][s[2*i]]
        ret_flight = flights[(destination, origin)][s[2*i+1]]
        
        # todos esperan al último familiar en llegar
        total_wait += latest_arrival - get_minutes(out_flight[1])
        
        # todos llegan al aeropuerto al mismo tiempo y esperan su vuelo
        total_wait += get_minutes(ret_flight[0]) - earliest_departure
        
        # si el último en llegar a NY llega después del primero en
        # irse de NY se paga un día más de la renta del carro.
        # el costo de la renta por un día es independiente de la
        # solución.
        if latest_arrival > earliest_departure:
            total_price += 50
    
    # El costo total es el precio total de los vuelos y el tiempo de
    # espera total de las personas.
    # Buscamos soluciones con un bajo costo.
    return total_price + total_wait # Quizás dándole más peso a la espera y no al precio la solución sea más idonea... total_wait * 2...

In [242]:
%%timeit
lista_sol = []
for i in range(10):
    lista_sol.append(schedule_cost(random_sol()))

# print(min(lista_sol))
# print(max(lista_sol))

3.19 ms ± 330 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Veamos qué tan malo es nuestro ejemplo con estos criterios de costo:

In [243]:
people

[('Seymour', 'BOS'),
 ('Franny', 'DAL'),
 ('Zooey', 'CAK'),
 ('Walt', 'MIA'),
 ('Buddy', 'ORD'),
 ('Les', 'OMA')]

In [244]:
flights[('BOS', 'LGA')]

[('6:17', '8:26', 89),
 ('8:04', '10:11', 95),
 ('9:45', '11:50', 172),
 ('11:16', '13:29', 83),
 ('12:34', '15:02', 109),
 ('13:40', '15:37', 138),
 ('15:27', '17:18', 151),
 ('17:11', '18:30', 108),
 ('18:34', '19:36', 136),
 ('20:17', '22:22', 102)]

In [245]:
flights[('LGA', 'BOS')]

[('6:39', '8:09', 86),
 ('8:23', '10:28', 149),
 ('9:58', '11:18', 130),
 ('10:33', '12:03', 74),
 ('12:08', '14:05', 142),
 ('13:39', '15:30', 74),
 ('15:25', '16:58', 62),
 ('17:03', '18:03', 103),
 ('18:24', '20:49', 124),
 ('19:58', '21:23', 142)]

In [246]:
schedule_cost([1,4,3,2,7,3,6,3,2,4,5,3])

4885

## Buscar todas las soluciones

¡Perfecto! Ahora solo tenemos que considerar todas las combinaciones de vuelos para determinar cuál tiene el menor costo.

In [247]:
print(f"Hay {len(people)} personas en la familia:")
total_solutions = 1
for p in people:
    name = p[0]
    origin = airports[p[1]][2]
    outs = len(flights[(p[1],destination)])
    rets = len(flights[(destination,p[1])])
    total_solutions *= outs
    total_solutions *= rets
    print(f"- {name} parte de {origin}, tiene {outs} opciones de salida y {rets} opciones de regreso;")
print(f"Por lo que hay un total de {total_solutions} posibles soluciones que analizar!\n\n")

if total_solutions > 1e9:
    print("¡Caracoles! esas son muchas soluciones")
else:
    print("Está facilito encontrar la mejor")

Hay 6 personas en la familia:
- Seymour parte de Boston, tiene 10 opciones de salida y 10 opciones de regreso;
- Franny parte de Dallas, tiene 10 opciones de salida y 10 opciones de regreso;
- Zooey parte de Akron, tiene 10 opciones de salida y 10 opciones de regreso;
- Walt parte de Miami, tiene 10 opciones de salida y 10 opciones de regreso;
- Buddy parte de Chicago, tiene 10 opciones de salida y 10 opciones de regreso;
- Les parte de Omaha, tiene 10 opciones de salida y 10 opciones de regreso;
Por lo que hay un total de 1000000000000 posibles soluciones que analizar!


¡Caracoles! esas son muchas soluciones


![GIF dolor](https://media.giphy.com/media/7T33BLlB7NQrjozoRB/giphy.gif)

## Buscar aleatoriamente

Veamos si podemos encontrar un buen resultado haciendo una búsqueda aleatoria en el espacio de soluciones.

![GIF random](https://media.giphy.com/media/89Eko49m84Ja/giphy.gif)

Consideremos la función `solve_randomly` que toma dos parámetros:
1. El *dominio* que consiste en una secuencia de tuplas `(min, max)` que establecen el valor mínimo y máximo que pueden tomar las entradas de las soluciones (de esta forma, codificamos el espacio de posibles soluciones de manera sucinta)
2. Una función de *costo*, que toma una posible solución y nos regresa un valor numérico que queremos minimizar.

Es importante observar que la cantidad de elementos en el dominio es igual a la cantidad de elementos en una solución.

Vamos a generar de forma aleatoria soluciones y regresar aquella con el costo mas pequeño.

In [248]:
sol = [1,4,3,2,7,3,6,3,2,4,5,3]

In [249]:
people[0]

('Seymour', 'BOS')

In [250]:
people[1]

('Franny', 'DAL')

In [251]:
flights[('LGA', 'MIA')]

[('6:33', '9:14', 172),
 ('8:23', '11:07', 143),
 ('9:25', '12:46', 295),
 ('11:08', '14:38', 262),
 ('12:37', '15:05', 170),
 ('14:08', '16:09', 232),
 ('15:23', '18:49', 150),
 ('16:50', '19:26', 304),
 ('18:07', '21:30', 355),
 ('20:27', '23:42', 169)]

In [252]:
dom = [(0,9)]*12
dom

[(0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9)]

In [253]:
def random_solution(domain):
    return [
        random.randint(r[0], r[1])
        for r in domain
    ]

In [254]:
random_solution(dom)

# schedule_cost(random_solution(dom))

[0, 1, 5, 1, 2, 4, 6, 9, 8, 1, 8, 5]

In [255]:
def solve_randomly(domain, cost_of, repeats = 1000):
    best_cost = float('inf')
    best_sol = None
    
    for _ in range(repeats):
        s = random_solution(domain)
        c = cost_of(s)
        if c < best_cost:
            best_cost = c
            best_sol = s
    
    return best_sol

In [256]:
(10*10)**6

1000000000000

In [257]:
domain = [(0,9)] * len(people) * 2

In [258]:
def test_randomly(repeats = 1000):
    s = solve_randomly(
        domain,
        schedule_cost,
        repeats
    )
    print_schedule(s)
    print(f"\nCon costo {schedule_cost(s)}")

In [259]:
test_randomly(100000)

   Seymour    Boston 11:16-13:29 $ 83 10:33-12:03 $ 74
    Franny    Dallas 10:30-14:57 $290  9:49-13:51 $229
     Zooey     Akron 12:08-14:59 $149 10:32-13:16 $139
      Walt     Miami  9:15-12:29 $225  9:25-12:46 $295
     Buddy   Chicago 14:22-16:32 $126 12:08-14:47 $231
       Les     Omaha 15:03-16:42 $135 11:07-13:24 $171

Con costo 3535


In [260]:
%timeit -n 1 -r 1 test_randomly()

   Seymour    Boston 12:34-15:02 $109 17:03-18:03 $103
    Franny    Dallas 10:30-14:57 $290 14:20-17:32 $332
     Zooey     Akron 12:08-14:59 $149 10:32-13:16 $139
      Walt     Miami 12:05-15:30 $330 11:08-14:38 $262
     Buddy   Chicago  8:25-10:34 $157 12:08-14:47 $231
       Les     Omaha 15:03-16:42 $135 12:31-14:02 $234

Con costo 4389
351 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [261]:
%timeit -n 1 -r 1 test_randomly(int(1e4))

   Seymour    Boston 12:34-15:02 $109  8:23-10:28 $149
    Franny    Dallas 10:30-14:57 $290  9:49-13:51 $229
     Zooey     Akron 10:53-13:36 $189  8:19-11:16 $122
      Walt     Miami 11:28-14:40 $248 11:08-14:38 $262
     Buddy   Chicago 12:44-14:17 $134 10:33-13:11 $132
       Les     Omaha 13:37-15:08 $250  8:04-10:59 $136

Con costo 3210
3.07 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [262]:
%timeit -n 1 -r 1 test_randomly(int(2e4))

   Seymour    Boston 12:34-15:02 $109 13:39-15:30 $ 74
    Franny    Dallas 12:19-15:25 $342 12:20-16:34 $500
     Zooey     Akron 12:08-14:59 $149 13:37-15:33 $142
      Walt     Miami 11:28-14:40 $248 12:37-15:05 $170
     Buddy   Chicago  9:42-11:32 $169 17:06-20:00 $ 95
       Les     Omaha 13:37-15:08 $250 16:35-18:56 $144

Con costo 3750
5.92 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [263]:
100*int(2e4) / ((10*10)**6)

2e-06

## Pensando en nuestros alrededores


Intentar obtener un buen resultado generando soluciones aleatorias es una ineficiente y pésima estrategia en este caso.

Un problema obvio de el método es que no aprovecha la información de las mejores soluciones que ha generado para generar otras buenas soluciones.

En nuestro problema particular, una solución con bajo costo es probablemente similar a otras soluciones con bajo costo.

Vamos a incorporar esta idea implementando en Python un método alternativo llamado *descenso de colinas*. Comenzamos con una solución aleatoria y buscamos en la *vecindad* de la solución por aquellas que mejoran el costo.

![GIF sisifo](https://media.giphy.com/media/xT0BKumCMrUb0dCypa/giphy.gif)

Detendremos la búsqueda hasta llegar a una solución cuya vecindad no mejora el costo.

In [264]:
def neighbors_of(s, domain):
    neighbors = []
    for i in range(len(domain)):
        if s[i] > domain[i][0]:
            neighbors.append(s[0:i] + [s[i] - 1] + s[i+1:])
            #print("Mayor que: ", i)
            #print(neighbors)
        if s[i] < domain[i][1]:
            neighbors.append(s[0:i] + [s[i] + 1] + s[i+1:])
            #print("Menor que: ", i)
            #print(neighbors)
    return neighbors

In [265]:
neighbors_of([3,0,0,0,0,0,0,0,0,0,0,9], domain)

[[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [3, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 9],
 [3, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 9],
 [3, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 9],
 [3, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 9],
 [3, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 9],
 [3, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 9],
 [3, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 9],
 [3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 9],
 [3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8]]

In [266]:
len(domain)
domain[2][1]

9

Este criterio de vecindad corresponde a todas las soluciones que están a distancia 1 (de acuerdo a Hamming).

Veamos la lista de vecinos de nuestra solución de ejemplo:

In [267]:
neighbors_of([1,4,3,2,7,3,6,3,2,4,5,3], domain)

[[0, 4, 3, 2, 7, 3, 6, 3, 2, 4, 5, 3],
 [2, 4, 3, 2, 7, 3, 6, 3, 2, 4, 5, 3],
 [1, 3, 3, 2, 7, 3, 6, 3, 2, 4, 5, 3],
 [1, 5, 3, 2, 7, 3, 6, 3, 2, 4, 5, 3],
 [1, 4, 2, 2, 7, 3, 6, 3, 2, 4, 5, 3],
 [1, 4, 4, 2, 7, 3, 6, 3, 2, 4, 5, 3],
 [1, 4, 3, 1, 7, 3, 6, 3, 2, 4, 5, 3],
 [1, 4, 3, 3, 7, 3, 6, 3, 2, 4, 5, 3],
 [1, 4, 3, 2, 6, 3, 6, 3, 2, 4, 5, 3],
 [1, 4, 3, 2, 8, 3, 6, 3, 2, 4, 5, 3],
 [1, 4, 3, 2, 7, 2, 6, 3, 2, 4, 5, 3],
 [1, 4, 3, 2, 7, 4, 6, 3, 2, 4, 5, 3],
 [1, 4, 3, 2, 7, 3, 5, 3, 2, 4, 5, 3],
 [1, 4, 3, 2, 7, 3, 7, 3, 2, 4, 5, 3],
 [1, 4, 3, 2, 7, 3, 6, 2, 2, 4, 5, 3],
 [1, 4, 3, 2, 7, 3, 6, 4, 2, 4, 5, 3],
 [1, 4, 3, 2, 7, 3, 6, 3, 1, 4, 5, 3],
 [1, 4, 3, 2, 7, 3, 6, 3, 3, 4, 5, 3],
 [1, 4, 3, 2, 7, 3, 6, 3, 2, 3, 5, 3],
 [1, 4, 3, 2, 7, 3, 6, 3, 2, 5, 5, 3],
 [1, 4, 3, 2, 7, 3, 6, 3, 2, 4, 4, 3],
 [1, 4, 3, 2, 7, 3, 6, 3, 2, 4, 6, 3],
 [1, 4, 3, 2, 7, 3, 6, 3, 2, 4, 5, 2],
 [1, 4, 3, 2, 7, 3, 6, 3, 2, 4, 5, 4]]

In [268]:
neighbors_of([6,0,0,0,0,0,6,0,0,0,0,0], domain)

[[5, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0],
 [7, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0],
 [6, 1, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0],
 [6, 0, 1, 0, 0, 0, 6, 0, 0, 0, 0, 0],
 [6, 0, 0, 1, 0, 0, 6, 0, 0, 0, 0, 0],
 [6, 0, 0, 0, 1, 0, 6, 0, 0, 0, 0, 0],
 [6, 0, 0, 0, 0, 1, 6, 0, 0, 0, 0, 0],
 [6, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0],
 [6, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0],
 [6, 0, 0, 0, 0, 0, 6, 1, 0, 0, 0, 0],
 [6, 0, 0, 0, 0, 0, 6, 0, 1, 0, 0, 0],
 [6, 0, 0, 0, 0, 0, 6, 0, 0, 1, 0, 0],
 [6, 0, 0, 0, 0, 0, 6, 0, 0, 0, 1, 0],
 [6, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 1]]

In [269]:
# Revisar este método...

def neighbour_of(s, domain):
    neighbours = []
    for i, x in enumerate(s):
        for j in set(range(domain[i][0], domain[i][1] + 1)) - {x}:
            neighbours.append(s[:i] + [j] + s[i + 1:])
    return neighbours

In [270]:
neighbour_of([3,0,0,0,0,0,0,0,0,0,0,9], domain)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [3, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [3, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [3, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [3, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [3, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [3, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [3, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [3, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [3, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [3, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 9],
 [3, 0, 8, 0, 0, 0, 0, 0,

In [271]:
def solve_hillclimbing(domain, cost_of):
    s = random_solution(domain)

    while True:
        neighbors = neighbors_of(s, domain)
        cost = cost_of(s)
        best_neighbor = min(neighbors, key=cost_of)
        neighbor_cost = cost_of(best_neighbor)
        
        if cost < neighbor_cost:
            return s
        
        s = best_neighbor


In [272]:
def test_hillclimbing():
    s = solve_hillclimbing(
        domain,
        schedule_cost,
    )
    print_schedule(s)
    print(f"\nCon costo {schedule_cost(s)}")

In [273]:
test_hillclimbing()

   Seymour    Boston  9:45-11:50 $172  8:23-10:28 $149
    Franny    Dallas  6:12-10:22 $230  9:49-13:51 $229
     Zooey     Akron  8:27-10:45 $139  8:19-11:16 $122
      Walt     Miami  9:15-12:29 $225  8:23-11:07 $143
     Buddy   Chicago  9:42-11:32 $169  7:50-10:08 $164
       Les     Omaha  9:15-12:03 $ 99  8:04-10:59 $136

Con costo 2858


In [274]:
%timeit -n 1 -r 1 test_hillclimbing()

   Seymour    Boston 17:11-18:30 $108 10:33-12:03 $ 74
    Franny    Dallas 10:30-14:57 $290 10:51-14:16 $256
     Zooey     Akron 17:08-19:08 $262 10:32-13:16 $139
      Walt     Miami 11:28-14:40 $248 12:37-15:05 $170
     Buddy   Chicago 15:58-18:40 $173 10:33-13:11 $132
       Les     Omaha 16:51-19:09 $147 11:07-13:24 $171

Con costo 3241
203 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Calentando los motores

Un problema del método de descenso de colinas es que una vez que se llega a un mínimo local, se deja de buscar. Sin embargo, este mínimo local puede corresponder a una pésima solución.

Podemos perturbar la búsqueda con un método llamado *recocido simulado*.

![GIF forjar](https://media.giphy.com/media/NpILbqtmLO1Qkfvc4f/giphy.gif)

Este método consiste en partir de un estado caliente, en donde es probable seguir un vecino con peor costo. Poco a poco, la temperatura del sistema baja, de tal forma que cada vez es menos probable elegir un peor vecino. Eventualmente, la temperatura es suficientemente baja como para detener la búsqueda.

Esto nos permite introducir una probabilidad variable para evitar algunos mínimos locales.

La estrategia es la siguiente: comenzamos con una solución aleatoria y una temperatura alta, tomamos un vecino aleatorio y analizamos los casos:
1. Si el vecino tiene menor costo $c'$ que la solución actual con costo $c$, elegimos al vecino como nueva solución
2. Si no es el caso, elegimos al vecino como nueva solución con probabilidad $$\mathrm{e}^{(c-c')/T}$$

In [275]:
4**1/2

2.0

In [276]:
def solve_annealing(domain, cost_of, Ti=10000.0, Tf=0.1, alpha=0.95):
    solution = random_solution(domain)
    cost = cost_of(solution)
    T = Ti
    while T > Tf:
        neighbor = random.choice(neighbors_of(solution, domain))
        neighbor_cost = cost_of(neighbor)
        diff = cost - neighbor_cost
        if diff > 0 or random.random() < math.exp(diff / T):
            solution = neighbor
            cost = neighbor_cost
        T = alpha*T
    
    return solution

In [277]:
def test_annealing(Ti=100000.0, Tf=0.1, alpha=0.95):
    s = solve_annealing(
        domain,
        schedule_cost,
        Ti, Tf, alpha,
    )
    print_schedule(s)
    print(f"\nCon costo {schedule_cost(s)}")

In [278]:
%timeit -n 1 -r 1 test_annealing(Ti=10000.0, Tf=0.1, alpha=0.95)

   Seymour    Boston 12:34-15:02 $109 10:33-12:03 $ 74
    Franny    Dallas 10:30-14:57 $290 10:51-14:16 $256
     Zooey     Akron 12:08-14:59 $149 10:32-13:16 $139
      Walt     Miami 11:28-14:40 $248 12:37-15:05 $170
     Buddy   Chicago 12:44-14:17 $134 10:33-13:11 $132
       Les     Omaha 12:18-14:56 $172 15:07-17:21 $129

Con costo 2804
88 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [279]:
%timeit -n 1 -r 1 test_annealing(Ti=10000.0, Tf=0.1, alpha=0.99)

   Seymour    Boston 18:34-19:36 $136 15:25-16:58 $ 62
    Franny    Dallas 18:26-21:29 $464 17:14-20:59 $277
     Zooey     Akron 18:35-20:28 $204 15:50-18:45 $243
      Walt     Miami 18:23-21:35 $134 15:23-18:49 $150
     Buddy   Chicago 18:48-21:45 $246 15:04-17:23 $189
       Les     Omaha 12:18-14:56 $172 15:07-17:21 $129

Con costo 3566
369 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [280]:
%timeit -n 1 -r 1 test_annealing(Ti=100000.0, Tf=0.1, alpha=0.99)

   Seymour    Boston 12:34-15:02 $109  8:23-10:28 $149
    Franny    Dallas 10:30-14:57 $290  9:49-13:51 $229
     Zooey     Akron 12:08-14:59 $149 13:37-15:33 $142
      Walt     Miami 11:28-14:40 $248  8:23-11:07 $143
     Buddy   Chicago 12:44-14:17 $134  7:50-10:08 $164
       Les     Omaha 12:18-14:56 $172  8:04-10:59 $136

Con costo 2992
438 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [281]:
%timeit -n 1 -r 1 test_annealing(Ti=1000000.0, Tf=0.1, alpha=0.99)

   Seymour    Boston  9:45-11:50 $172 10:33-12:03 $ 74
    Franny    Dallas  6:12-10:22 $230 10:51-14:16 $256
     Zooey     Akron  8:27-10:45 $139 13:37-15:33 $142
      Walt     Miami  9:15-12:29 $225 12:37-15:05 $170
     Buddy   Chicago  9:42-11:32 $169 10:33-13:11 $132
       Les     Omaha  9:15-12:03 $ 99 11:07-13:24 $171

Con costo 2992
486 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [282]:
%timeit -n 1 -r 1 test_annealing(Ti=1000000.0, Tf=0.1, alpha=0.999)

   Seymour    Boston 13:40-15:37 $138 15:25-16:58 $ 62
    Franny    Dallas 10:30-14:57 $290 17:14-20:59 $277
     Zooey     Akron 13:40-15:38 $137 15:50-18:45 $243
      Walt     Miami 11:28-14:40 $248 15:23-18:49 $150
     Buddy   Chicago 14:22-16:32 $126 15:04-17:23 $189
       Les     Omaha 15:03-16:42 $135 15:07-17:21 $129

Con costo 3009
4.96 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## ¿Evolución al rescate?

![GIF darwin](https://media.giphy.com/media/VFAke5Xm1TDwjgimyW/giphy.gif)

Ahora vamos a considerar otro método, que al igual que el recocido simulado está inspirado en la naturaleza, pero no en la física, si no en la biología.

Primero se crea un conjunto de soluciones aleatorias, conocidas como *la población*. En cada paso del método, la función de costo para cada solución en la población es calculada, esto nos permite ordenar las soluciones de mejor a peor.

Posteriormente, una nueva población es generada a partir de la actual: Las mejores soluciones de la actual se conservan tal cuál (*elitismo*). El resto de la población, va a ser modificada para obtener la nueva población.

Hay dos formas en que las soluciones pueden ser modificadas. La mas simple es llamada *mutación* y consiste en un pequeño y simple cambio aleatorio a la solución actual. Esto sería similar a elegir un vecino de forma aleatoria.

Otra manera de modificar las soluciones es llamado *cruza*, consiste en tomar dos soluciones de las mejores y combinarlas de alguna manera. Una forma simple de combinarlas es tomar una cantidad aleatoria de elementos de una buena solución y acompletar los elementos que faltan con otra buena solución.

Una vez que se obtiene la nueva población, el proceso continúa.

In [283]:
def mutate(s, domain):
    return random.choice(neighbors_of(s, domain))

In [284]:
def crossover(s1, s2):
    i = random.randint(1, len(s1)-2)
    return s1[:i] + s2[i:]

In [285]:
def solve_evolving(domain, cost_of, pop_size=50, mut_prob=0.2, elite=0.2, epochs=100):
    pop = [random_solution(domain) for _ in range(pop_size)]
    top_elite = int(elite * pop_size)
    
    for epoch in range(epochs):
        pop.sort(key=cost_of)
        best = pop[:top_elite]
        while len(best) < pop_size:
            if random.random() < mut_prob:
                best.append(mutate(
                    best[random.randint(0, top_elite-1)],
                    domain
                ))
            else:
                best.append(crossover(
                    best[random.randint(0, top_elite-1)],
                    best[random.randint(0, top_elite-1)],
                ))
        pop = best
    pop.sort(key=cost_of)
    return pop[0]

In [286]:
def test_evolving(pop_size=50, mut_prob=0.2, elite=0.2, epochs=100):
    s = solve_evolving(
        domain,
        schedule_cost,
        pop_size, mut_prob,
        elite, epochs,
    )
    print_schedule(s)
    print(f"\nCon costo {schedule_cost(s)}")

In [287]:
%timeit -r 1 -n 1 test_evolving(50, 0.2, 0.2, 100)

   Seymour    Boston 20:17-22:22 $102 13:39-15:30 $ 74
    Franny    Dallas 18:26-21:29 $464 17:14-20:59 $277
     Zooey     Akron 18:35-20:28 $204 13:37-15:33 $142
      Walt     Miami 19:53-22:21 $173 15:23-18:49 $150
     Buddy   Chicago 19:50-22:24 $269 14:19-17:09 $190
       Les     Omaha 16:51-19:09 $147 15:07-17:21 $129

Con costo 3449
1.28 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [288]:
%timeit -r 1 -n 1 test_evolving(50, 0.2, 0.2, 200)

   Seymour    Boston 17:11-18:30 $108 10:33-12:03 $ 74
    Franny    Dallas 13:54-18:02 $294 10:51-14:16 $256
     Zooey     Akron 17:08-19:08 $262 10:32-13:16 $139
      Walt     Miami 11:28-14:40 $248 15:23-18:49 $150
     Buddy   Chicago 15:58-18:40 $173 10:33-13:11 $132
       Les     Omaha 16:51-19:09 $147 11:07-13:24 $171

Con costo 3206
2.76 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [289]:
%timeit -r 1 -n 1 test_evolving(200, 0.2, 0.2, 100)

   Seymour    Boston 17:11-18:30 $108  8:23-10:28 $149
    Franny    Dallas 13:54-18:02 $294  9:49-13:51 $229
     Zooey     Akron 15:23-17:25 $232  8:19-11:16 $122
      Walt     Miami 15:34-18:11 $326  8:23-11:07 $143
     Buddy   Chicago 15:58-18:40 $173  7:50-10:08 $164
       Les     Omaha 15:03-16:42 $135  8:04-10:59 $136

Con costo 3009
5.32 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# La tarea

1. Modela este problema utilizando técnicas de orientado a objetos.
2. Explora los parámetros y la función costo de este problema, intenta encontrar mejores valores para resolver el problema.

Explorados...

3. Elige un problema distinto, y modela las posibles soluciones del problema para utilizar los métodos discutidos en esta libreta: búsqueda de solución aleatoria, búsqueda de solución por descenso de colinas, búsqueda de solución por recocido simulado, búsqueda de solución por algoritmos genéticos.
4. Compara los resultados de las soluciones de los cuatro métodos anteriores.

Comparados...

5. ¿Es posible resolver el problema que planteas analizando todas las posibles soluciones? Justifica tu respuesta.

No. Porque el dominio de posibles soluciones es muy extenso pero con el uso de la función de costo se puede encontrar la mejor solución posible para el tiempo conque se cuenta.

In [290]:
#Estructura de Datos... sucintos???

def binrep(n):
    rep = ""
    while n > 0:    
        r = n % 2
        n = n // 2
        rep = str(r) + rep
    return rep
    #print(r, q)

binrep(320)


'101000000'

In [291]:
bin(320)

'0b101000000'

In [292]:
a= exec("0b" + "101000000")

print(a)

None


In [293]:
# Eval ----> Interpretar una cadena de Python...
eval("0b" + "101000000")


320

In [294]:
# Se pudiera convertir un Binario a Entero... cosa que no se puede hacer con
#int("0b" + "101000000") # Usar Eval...

In [295]:
eval?


Signature: eval(source, globals=None, locals=None, /)
Docstring:
Evaluate the given source in the context of globals and locals.

The source may be a string representing a Python expression
or a code object as returned by compile().
The globals must be a dictionary and locals can be any mapping,
defaulting to the current globals and locals.
If only globals is given, locals defaults to it.
Type:      builtin_function_or_method

In [296]:
exec?

Signature: exec(source, globals=None, locals=None, /, *, closure=None)
Docstring:
Execute the given source in the context of globals and locals.

The source may be a string representing one or more Python statements
or a code object as returned by compile().
The globals must be a dictionary and locals can be any mapping,
defaulting to the current globals and locals.
If only globals is given, locals defaults to it.
The closure must be a tuple of cellvars, and can only be used
when source is a code object requiring exactly that many cellvars.
Type:      builtin_function_or_method

In [ ]:
class paciente:
    """Paciente al cual se le realizará un diagnóstico X, y este será su estado actual, siendo la meta encontrar el mejor tratamiento en un dominio Y de opciones..."""
    
    def __init__(self, dx_actual, sano = False):
        """Constructor inicializado..."""
        self.dx_actual = dx_actual
        self.sano = sano
    
    def actions(self, state):
        """Tipos de tratamientos disponibles"""
        lista_de_acciones = find_acciones(state) # Crear esta función...
        return lista_de_acciones
    
    def result(self, state, action):
        """Da el resultado de aplicar una action de la lista de acciones..."""
        # Cambiar el estado actual después de aplicar tratamiento...
        state = aplica_tratamiento(state, action) # Crear función...
        return state
        
    def cheaca_salud(self, state):
        """Retornar True si el state pertenece a una lista donde se considere que ya el paciente está Sano..."""
        if isinstance(state, list_sano):
            self.sano = True
            return "Sano"
        return state
    
    def path_cost(self, c, state1, action, state2):
        """Crear un método para calcular el costo que llevaría mejorar."""
        # Crear una función de costo...
        return "costoso... muy costoso..."
    
    def codigo_CID(self, state):
        """Devuelve el código CID-10 de un diagnóstico X..."""
        cid10 = codificar(state)
        return cid10

Que la función de costo venga del constructor...


Para cada persona que  costoso será segun cada tipo de costo.


Ponderarlo... para ponderarlo necesito saber como las personas eligen opciones, que aspectos toman en concideración de forma personal, cuales en si les conviene tomar según su estado de salud anterior...


Dinero  0.2
Sentimental   0.5
Tiempo 0.1
Físico.   0.2
Valores que sumen 1.0


Le calculo el costo a una tupla (Diagnóstico, acción) ... pero se pudiera tener en cuenta también APP, APF, Diagnóstico, Acción...


Recibir este vector segun cada paciente.


Los vecinos serían estar en un diagnóstico y tomar una acción...


Como modelar la experiencia del panciente con salud...


Orientado a objeto como modelar estas acciones...


Criterios para saber el estado inicial...
list_actions...


Como modelar las clases si en una misma consulta se diagnostican 2 enfermedades...
En este caso se sobrecargaría el operador sumar por ejemplo, y la solución sería digamos un pronmedios de las soluciones... aunque... se adicionarían las contraindicaciones y las interacciones... entre las soluciones...


Ver simulaciones basadas en agantes...
Ver otros tipos de simulaciones...

Que otras clases se pueden modelar...
Descargar software de Pandemia... simulaciones... descargar códigos de pandemias...

Precisar como se ve un estado??? Como se ve una solución??? Como se ve una acción???
Son listas??? Son Tuplas??? Valores???
